In [ ]:

import os
from langchain.vectorstores.chroma import Chroma
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import NewsURLLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [ ]:
with open(r"data/CNN_Links.txt") as f:
    urls = [link.replace("\n", "") for link in set(f.readlines())]

loader = NewsURLLoader(urls=urls)
data = loader.load()
print("Primeira notícia: ", data[0])
print("\nSegunda notícia: ", data[1])


In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'data/db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

#vectordb = Chroma.from_texts([], embedding=embedding, persist_directory=persist_directory)
contents = [item.page_content for item in data]
sources = [{'source_info': item.metadata['link']} for item in data]

# Initialize Chroma without specifying texts initially
vectordb = Chroma.from_texts(contents, embedding, metadatas = sources, persist_directory=persist_directory)

vectordb.persist()

In [ ]:
#Declaração do modelo de embedding
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
persist_directory = 'data/db'
embedding = instructor_embeddings

#Criação do objeto de gestão da vector db
vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory)

retriever = vectordb.as_retriever()
query = input("The first question:  ")
#Retorna os 3 primeiros itens relevantes para a pergunta do usuário
context = retriever.get_relevant_documents(query)[:1]

In [ ]:
context